# Read Input & Display Wav

In [ ]:
from extras.FPP_functions import *
from extras.ReadFringeImages import *
import numpy as np
import matplotlib.pyplot as plt
import cv2;
from scipy.misc import imresize



import dtcwt

MaxIter = 1
offset = [500, 700]
sz = 768
J = 5

prefixFld = '../images/'
RecOri = ReadImage(prefixFld, 'bottle', 'rec_fringe_1', offset, sz)

X = RecOri[:,:,0]
transform = dtcwt.Transform2d(biort='near_sym_b', qshift='fjwang2')
transform2 = dtcwt.Transform2d()
J = 5
w = transform.forward(X, nlevels=5)
w2 = transform2.forward(X, nlevels=5)

# for j in range(0, J):
# #     imagesc(np.float32(w.highpasses[j][:,:,5]), dpi_=200)
#     imagesc(np.float32(w2.highpasses[j][:,:,5]), dpi_=200)
    


In [2]:



# Read image
prefixFld = '../images/'
RecOri = ReadImage(prefixFld, 'bottle', 'rec_fringe_1', offset, sz)

print("Rec Ori shape = ", RecOri.shape)

# imshow(RecOri)
# DTCWT
J = 5
w = DTCWT_Forw(RecOri[:,:,1], J)
print(768/2**5)
print(w.highpasses[0].shape)
# imagesc(np.float32(w.highpasses[0][:,:,0]), dpi_=200)
# imagesc(np.float32(w.highpasses[1][:,:,0]), dpi_=200)
# imagesc(np.float32(w.highpasses[2][:,:,0]), dpi_=200)
# imagesc(np.float32(w.highpasses[3][:,:,0]), dpi_=200)
# imagesc(np.float32(w.highpasses[4][:,:,0]), dpi_=200)


../images/bottle/rec_fringe_1.jpg
Rec Ori shape =  (768, 768, 3)


NameError: name 'DTCWT_Forw' is not defined

# INITIALIZATION (Ground Truth(shape_GT) & mask)

In [ ]:
from extras.FPP_functions import *
from extras.ReadFringeImages import *

# Save to file
import scipy.io as sio
results = {}


N = 50
offset = [500, 700]
sz = 768


prefixFld = '../images/'
prefixImg = 'ref_fringe_'
fringe, phiRef_GT, flat = ReadFringeImages(prefixFld, 'Blk', prefixImg, N, offset, sz)
prefixFld = '../images/'
prefixImg2= 'rec_fringe_'
fringe, phiRec_GT, flat = ReadFringeImages(prefixFld, 'bottle', prefixImg2, N, offset, sz)


# FOR MASK 1
mask = logical_and(logical_and(flat[:, :, 0] > 80, flat[:, :, 1] > 80), flat[:, :, 2] > 0)

# FOR MASK 2
# maskI = ReadImage(prefixFld, '6_head2', 'ref_flatmask', offset, sz)
# # maskI = ReadImage(prefixFld, '6_jug', 'ref_flat', offset, sz)
# maskI = rgb2gray(maskI) 
# mask = maskI > 100

# FOR MASK Bottle
mask = logical_or(logical_and(flat[:, :, 0] > 50, flat[:, :, 1] > 50), flat[:, :, 2] > 50)



shape_GT = (unwrapGold(phiRec_GT, mask) - unwrapGold(phiRef_GT, mask) + 5*np.pi) * mask

results['shape_GT'] = shape_GT
results['mask'] = mask

shape_GT = np.multiply(shape_GT, mask)
shape_GT = shape_GT * (shape_GT < 20)

# imagesc(shape_GT, mask, 100)
# imagesc(mask, mask, 100)

print("Finished")


# Main process 

In [ ]:
import numpy as np

from models.model_wav import _netG
from models.model_wav_gray import _netGGray
from models.model_wav import *
import timeit
from torch.autograd import Variable
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import transforms
from extras.FPP_functions import *
from sklearn.feature_extraction import image
from skimage.restoration import denoise_tv_chambolle
import cv2
import prox_tv as ptv

from DnnFPP.ImageLoader import WavWraper 

def convertFringe(fringe):
#     fringe2 = np.zeros((fringe.shape[0],fringe.shape[1],fringe.shape[2]*2))
#     for c in range(0,6):
#         fringe2[:,:,c*2] = fringe[:,:,c].real
#         fringe2[:,:,c*2+1] = fringe[:,:,c].imag
    fringe2 = np.abs(fringe)    
    return fringe2
        

def processCNN(model, fringe):
    start_time = timeit.default_timer()
    # Start Running
    
    
    
    fringe = fringe.reshape((1, fringe.shape[2], fringe.shape[0], fringe.shape[1]))
    print(fringe.shape, type(fringe))
    Rec = model(Variable(torch.from_numpy(fringe).cuda()))
    Rec = Rec.data.cpu().numpy()
    print(Rec.shape)
    Rec = np.transpose(Rec, (0, 2, 3, 1))

    print(Rec.shape)
    Rec = Rec.reshape(Rec.shape[1], Rec.shape[2], Rec.shape[3])
    elapsed = timeit.default_timer() - start_time
    print('total time = {} ms'.format(elapsed* 1000) )
    return Rec

def adjustShape(shape_CNN, shape_GT, r=600, c=400):    
    delta = shape_CNN[r,c]-shape_GT[r,c]
    print("delta ", delta)
    shape_CNN = np.subtract(shape_CNN, delta)
    return shape_CNN

def momTh(I):    
    nChannel = I.shape[2]
    
    th = np.zeros(3)
    for c in range(0,nChannel):
        tmp = np.float32(I[:,:,c])
        dctI = cv2.dct(tmp)
        th[c] = np.amax(dctI)
    return th
def momTh2(I): 
    print("Mom2 = ", I.shape)    
    nChannel = I.shape[2]    
    th = np.zeros(3)
    for c in range(0, nChannel):
        LSet, HSet, maxTh2 = getWaveFrIGray(I[:,:,c], J)
        th[c] = maxTh2
    return th



        
# INITIALIZATION FOR PARAMETER
J = 4                        #  wavelet max Level
# nMaxChannel = J*3*3          # input and output channel of the wavelet transform (MAKE SURE IT IS THE SAME AS THE TRAINING)
# nMaxChannelGray = J*3          # input and output channel of the wavelet transform (MAKE SURE IT IS THE SAME AS THE TRAINING)
nMaxChannelDTCWT = 18

# LOAD THE SAVED TRAINED NETWORK
model = _netG(BasicBlock, BasicBlockUp, BasicBlockDown, nMaxChannelDTCWT, nMaxChannelDTCWT)
lr= 0.01
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=lr)  #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, dampening=0, weight_decay=0.0001)
# IF NOT MATCH CHECK THE MODEL CONSTRUCTOR
checkpoint = torch.load('cp_wav_29_0.001_16.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])



# Read first fringe index = 0
RecOri = ReadImage(prefixFld, 'bottle', 'rec_fringe_1', offset, sz)
imshow(RecOri.astype(np.uint8), dpi_=200)  

WWrap = WavWraper()
J = 4
for c in range(0,3):
    X = RecOri[:,:,c].astype(np.float32)
    w = WWrap.DTCWT_F(X, J)
    if c == 0:
        HSet = convertFringe(w.highpasses[2]).astype(np.float32)
    else:
        tmp = convertFringe(w.highpasses[2]).astype(np.float32)
        HSet = np.concatenate((HSet, tmp), axis = 2)
       
print("Size HSet ", HSet.shape)
tmp = processCNN(model, HSet)

print("size tmp = ", tmp.shape)

result = RecOri
for c in range(0,3):
    X = RecOri[:,:,c].astype(np.float32)
    w = WWrap.DTCWT_F(X, J)
    
    agl = np.angle(w.highpasses[2])        
    a = tmp[:,:,6*c:6*c+6] * np.cos(agl)
    b = tmp[:,:,6*c:6*c+6] * np.sin(agl)    
    a = a + 1j * b
    w.highpasses[2][:,:,:] = a
    
    result[:,:,c] = WWrap.DTCWT_I(w)
    
#     w.highpasses[2]
    
  
imshow(result.astype(np.uint8), dpi_=200)    
    
# ###################### REDUCE ##########################
# tmp = RecOri
# mnRecOri = np.tile(np.mean(tmp, axis = 2).reshape(tmp.shape[0],tmp.shape[1],1), (1,1,3))
# fringe = np.subtract(tmp, mnRecOri)


# ### INITIALIZATION
# maxIter = 1
# X = RecOri.astype(np.float32)
# Xn = X
# Xt = np.zeros(X.shape)
# Xr = Xt

# ### WAVELET
# LSet, HSet, maxTh2 = getWaveFrI(X, J)
# wI = np.transpose(HSet, (2,0,1))
# wI = wI.reshape((1, wI.shape[0], wI.shape[1], wI.shape[2])).astype(np.float32)
# wRes = processCNN(model, wI)    
# Xn = getIFrWave(LSet, wRes, J)    


# # tmp = getIFrWave(LSet*0, HSet, J) 
# tmp = getIFrWave(np.zeros(LSet.shape), wRes, J)   # without DC component
# thDCT = momTh(tmp)
# maxThDCT = thDCT



    


# shape_CNN = (-unwrapGold(phiRec, mask)-unwrapGold(phiRef_GT, mask) + 2*np.pi) 
# shape_CNN = adjustShape(shape_CNN, shape_GT, r=400, c=350)
# results['shape_CNN'] = shape_CNN
# shape_CNN = shape_CNN * mask

# SNR, MSE, mseShape = CalcSNR_MSE(shape_CNN, shape_GT, mask)

    
    
# # imagesc(shape_GT, mask, dpi_=150, mn=0, mx=24, cmap_='gray')
# imagesc(shape_CNN, mask, dpi_=150, mn=0, mx=24, cmap_='gray')
# imshow(IRes.astype(np.uint8), dpi_=150)


# SNR, MSE, mseShape = CalcSNR_MSE(shape_CNN, shape_GT, mask)

# # imagesc(mseShape, mask, dpi_=150, mn=0, mx=2, cmap_='gray')

# plot(IRes[:,300,0], dpi_=200)
# plot(IRes[:,300,1], dpi_=200)
# plot(IRes[:,300,2], dpi_=200)
# # plot(residue[:,300], dpi_=200)
# # plot(RecOri[:,300,1], dpi_=200)


# plt.show()
# sio.savemat('result/gt.mat', results)

In [ ]:
from skimage import io
from sklearn.feature_extraction import image
import matplotlib.image as mpimg
import numpy as np
from scipy.signal import hilbert 

imgFld = '../images'
prefix = ['Blk', 'chkboard', 'bottle', 'flower']        
NN     = [50, 100, 100, 100] 

idx = 2
N = 100
filePrefix = prefix[idx] + '/' + prefix[idx]  + '_fringe_'
for i in range(N):
    filename = '{0}/{1}{2}.jpg'.format(imgFld, filePrefix, i+1)
    tmp = mpimg.imread(filename)
    
    if i == 0:
        img= tmp[offsetrow:offsetrow+sz,offsetcol:offsetcol+sz,:].astype(np.float32)
        fringe = img
    else:
        img = tmp[offsetrow:offsetrow+sz,offsetcol:offsetcol+sz,:].astype(np.float32) + img        
        
            

bias = img / N
print(bias.shape)

phi = np.subtract(fringe.astype(np.float32), bias);
phi = np.mean(phi, axis=2)

phi2 = hilbert(phi, axis=0)
aa = np.angle(phi2)

figure(1,dpi=200, facecolor='w', edgecolor='k')
plt.imshow(bias, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
a = np.ones((3,3)) * 2
print(np.prod(a.shape))
print(a)
a = a.reshape(1,a.shape[0], a.shape[1])
b = np.concatenate((a,a*2),axis=0)
print(b.shape)

print(b[1,:,:].shape)

In [ ]:
import torch

# x = -3
# y = 3
# z = x - y
# print(z)

# k = mod(z+3,6)-2.9
# print(k)
torch.__version__